# 훅을 사용한 Strands Agent와 AgentCore Memory 튜토리얼

## 개요

이 튜토리얼은 훅을 통해 AgentCore Memory와 통합된 Strands agents를 사용하여 지능형 개인 비서를 구축하는 방법을 보여줍니다. 에이전트는 대화 컨텍스트를 유지하고 상호작용에서 학습하여 개인화된 응답을 제공합니다.

## 튜토리얼 세부사항

**사용 사례**: 수학 비서

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | 장기 대화형                                                                         |
| 에이전트 유형        | 수학 비서                                                                         |
| 에이전트 프레임워크   | Strands Agents                                                                   |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                      |
| 튜토리얼 구성요소     | 메모리를 위한 AgentCore 요약 전략, 메모리 저장 및 검색을 위한 훅   |
| 예제 복잡도          | 중급                                                                             |


배울 내용:
- 대화 요약으로 AgentCore Memory 설정
- 자동 저장 및 검색을 위한 메모리 훅 생성
- 지속적인 메모리를 가진 Strands 에이전트 구축
- 대화 간 메모리 기능 테스트

### 시나리오 컨텍스트

이 예제에서는 이전 대화의 요약을 저장하는 수학 비서 예제를 만들어보겠습니다. 
이 예제의 주요 기능:
- **자동 메모리 저장**: 대화가 자동으로 저장됨
- **컨텍스트 검색**: 이전 대화가 현재 응답에 정보 제공
- **요약 생성**: 주요 정보가 추출되고 요약됨
- **도구 통합**: 수학 연산을 위한 계산기 도구

## 아키텍처
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## 전제조건

이 튜토리얼을 실행하려면 다음이 필요합니다:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 권한이 있는 AWS 자격 증명
- Amazon Bedrock AgentCore SDK

## 1단계: 환경 설정
이 노트북이 작동하도록 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것부터 시작해보겠습니다.

In [ ]:
!pip install -qr requirements.txt

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

In [ ]:
import os
import logging
from strands import Agent
from datetime import datetime
from strands_tools import calculator
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("memory-tutorial")

# Configuration - replace with your values
REGION = os.getenv('AWS_REGION', 'us-west-2')
ROLE_ARN = "<<INSERT-YOUR-IAM-ROLE>>"
ACTOR_ID = f"actor-{datetime.now().strftime('%Y%m%d%H%M%S')}"
SESSION_ID = f"tutorial-{datetime.now().strftime('%Y%m%d%H%M%S')}"

## 2단계: 메모리 리소스 생성

이 단계에서는 요약 전략을 가진 메모리 리소스를 생성합니다. 이 리소스는 대화 데이터를 저장하고 정리합니다. 우리가 정의하는 전략은 대화의 요약을 자동으로 생성하고 정리된 네임스페이스에 저장합니다.

먼저 수학 비서를 위한 사용자 정의 프롬프트를 만들어보겠습니다.


In [ ]:
CUSTOM_PROMPT = """
Your task is to extract math learning data from the user's conversations. You store the progress of the user in a memory system to understand their math level and help them progress.

You are tasked with analyzing conversations to extract the user's math learning patterns. You'll be analyzing two sets of data: 

<past_conversation> 
[Past conversations between the user and math tutor will be placed here for context] 
</past_conversation> 

<current_conversation> 
[The current conversation between the user and math tutor will be placed here] 
</current_conversation> 

Your job is to identify and categorize the user's math learning profile:
- Extract the user's current math level from problems they solve correctly/incorrectly
- Extract the user's preferred learning style from how they ask questions and respond to explanations
- Extract topic strengths and weaknesses from their performance patterns
- Track learning progress and identify areas needing reinforcement
"""

In [ ]:
from botocore.exceptions import ClientError

# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "MathAssistant"
# Define memory strategy for conversation summaries
strategies = [
    {
        StrategyType.CUSTOM.value: {
            "name": "CustomSemanticMemory",
            "description": "Captures facts from conversations",
            "namespaces": ["/students/math/{actorId}"],
            "configuration" : {
                "semanticOverride" : {
                    "extraction" : {
                        "modelId" : "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
                        "appendToPrompt": CUSTOM_PROMPT
                    }
                },
    }}}
]

# Create memory resource
try:
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=strategies, # Use the defined long term strategies
        description="Memory for tutorial agent",
        event_expiry_days=30,
        memory_execution_role_arn=ROLE_ARN,
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

## 3단계: 메모리 훅 프로바이더 생성

이 단계에서는 메모리 작업을 자동화하는 사용자 정의 `MemoryHookProvider` 클래스를 정의합니다. 훅은 에이전트의 실행 생명주기의 특정 지점에서 실행되는 특수 함수입니다. 우리가 만드는 메모리 훅은 두 가지 주요 기능을 제공합니다:

1. **메모리 검색**: 사용자가 메시지를 보낼 때 관련된 과거 대화를 자동으로 가져옴
2. **메모리 저장**: 에이전트가 응답한 후 새로운 대화를 저장

이는 수동 관리 없이 원활한 메모리 경험을 만들어줍니다.

In [ ]:
class MemoryHookProvider(HookProvider):
    """Hook provider for automatic memory management"""
    
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.namespace = f"/students/math/{self.actor_id}"
    
    def retrieve_memories(self, event: MessageAddedEvent):
        """Retrieve relevant memories before processing user message"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_message = messages[-1]["content"][0].get("text", "")
            
            try:
                # Retrieve relevant memories
                memories = self.client.retrieve_memories(
                    memory_id=self.memory_id,
                    namespace=self.namespace,
                    query=user_message
                )
                
                # Extract memory content
                memory_context = []
                for memory in memories:
                    if isinstance(memory, dict):
                        content = memory.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                memory_context.append(text)
                
                # Inject memories into user message
                if memory_context:
                    context_text = "\n".join(memory_context)
                    original_text = messages[-1]["content"][0].get("text", "")
                    messages[-1]["content"][0]["text"] = (
                        f"{original_text}\n\nPrevious context: {context_text}"
                    )
                    logger.info(f"Retrieved {len(memory_context)} memories")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve memories: {e}")
    
    def save_memories(self, event: AfterInvocationEvent):
        """Save conversation after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last user and assistant messages
                user_msg = None
                assistant_msg = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # Save conversation
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_memories)
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

## 4단계: 메모리를 가진 에이전트 생성

이제 Strands 에이전트를 생성하고 메모리 훅 프로바이더와 연결합니다. 이 에이전트는 두 가지 주요 기능을 가집니다:

1. **메모리 통합**: 우리가 만든 메모리 훅이 자동 컨텍스트 검색을 가능하게 함
2. **계산기 도구**: 에이전트가 필요할 때 수학 연산을 수행할 수 있음

이 조합은 과거 상호작용을 기억하고 유용한 계산을 수행할 수 있는 개인 비서를 만들어줍니다.

In [ ]:
# Create memory hook provider
memory_hooks = MemoryHookProvider(
    memory_id=memory_id,
    client=client,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID
)

# Create agent with memory hooks and calculator tool
agent = Agent(
    hooks=[memory_hooks],
    model = "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[calculator],
    system_prompt="You are a helpful personal math tutor. You assist users in solving math problems and provide personalized assistance."
)

print("✅ Agent created with memory hooks.")

**에이전트 설정이 완료되었습니다! 이제 테스트해보겠습니다.**

## 메모리 기능 테스트

이 섹션에서는 일련의 상호작용을 통해 에이전트의 메모리 기능을 테스트합니다. 에이전트가 시간이 지나면서 컨텍스트를 구축하고 이전 상호작용을 회상하는 방법을 관찰합니다.

먼저 에이전트에게 자기소개를 하고 수학 질문을 해보겠습니다:

In [ ]:
# First interaction - introduce yourself
response1 = agent("Hi, I'm John and I just enrolled in Discrete Math course. Help me solve this: How many ways can I arrange 5 books on a shelf?")
print(f"Agent: {response1}")

에이전트에게 다른 계산 작업을 주어보겠습니다:

In [ ]:
# Second interaction - another calculation
response2 = agent("I learn better with step-by-step explanation with example questions. Can you explain modular arithmetic? What's 17 mod 5?")
print(f"Agent: {response2}")

이제 에이전트가 우리가 누구인지 기억하는지 확인해보겠습니다.

**참고:** 메모리가 추출, 통합 및 저장될 시간을 주기 위해 약 20초 정도 일시 정지해주세요.

In [ ]:
# Third interaction - test memory recall
response3 = agent("I got that right! What's the immediate next step that I should study after modular arithmetic?")
print(f"Agent: {response3}")

마지막으로 에이전트가 우리의 계산 기록을 기억하는지 확인해보겠습니다:

In [ ]:
# Fourth interaction - test context awareness
response4 = agent("This is too hard, can we try something easier?")
print(f"Agent: {response4}")

### 메모리 저장 확인

마지막 단계로, 대화가 AgentCore Memory에 제대로 저장되었는지 확인합니다. 이는 메모리 훅이 올바르게 작동하고 에이전트가 향후 상호작용에서 이 정보에 액세스할 수 있음을 보여줍니다.

In [ ]:
# Check stored memories
try:
    memories = client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"/students/math/{ACTOR_ID}",
        query="mathematics calculations"
    )
    
    print(f"\n📚 Found {len(memories)} memories:")
    for i, memory in enumerate(memories, 1):
        if isinstance(memory, dict):
            content = memory.get('content', {})
            if isinstance(content, dict):
                text = content.get('text', '')[:200] + "..."
                print(f"{i}. {text}")
                
except Exception as e:
    print(f"Error retrieving memories: {e}")

튜토리얼 완료! 🎉

주요 요점:
- 메모리 훅이 대화 컨텍스트를 자동으로 저장하고 검색
- 에이전트가 여러 상호작용 간 상태를 유지할 수 있음
- AgentCore Memory가 관련 컨텍스트에 대한 시맨틱 검색 제공
- 도구를 메모리와 결합하여 향상된 기능 제공 가능

## 정리

### 선택사항: 메모리 리소스 삭제

튜토리얼을 완료한 후 불필요한 비용을 피하기 위해 메모리 리소스를 삭제할 수 있습니다. 다음 코드는 정리를 위해 제공되지만 기본적으로 주석 처리되어 있습니다.

In [ ]:
# Uncomment to delete the memory resource
# try:
#     client.delete_memory_and_wait(memory_id=memory_id)
#     print(f"✅ Deleted memory resource: {memory_id}")
# except Exception as e:
#     print(f"Error deleting memory: {e}")